In [ ]:
# importing dependencies 
from utils import *
from ddpms import *

In [ ]:
# defining directory and model_disct_state names
models_dicts = ["model_classic.pth", "model_lds_simple.pth", "model_lds_sobol.pth"]
gen_paths = ["./gen_classic", "./gen_lsd_simple","./gen_lsd_sobol"]
eval_path = "./eval_img"

In [ ]:
# Defining model parameter
T = 1000
learning_rate = 1e-3
epochs = 100
batch_size = 256
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
mnist_unet = ScoreNet((lambda t: torch.ones(1).to(device)))

In [ ]:
# Define dataloader for MNIST test set
dataloader_gen = torch.utils.data.DataLoader(
    datasets.MNIST(eval_path,
                   download=True,
                   train=False, # selecting test data
                   transform=transforms.ToTensor()),
    batch_size=batch_size,
    shuffle=True)

In [ ]:
FIDs = []

# Comparison loop
for state_dict,folder in zip(models_dicts, gen_paths):
    # Loading the model 
    if state_dict == "model_lds_simple.pth":
        model = DDPM_low_discrepancy(mnist_unet, T=T, sampler = "simple").to(device)
        model.load_state_dict(torch.load(state_dict))
    if state_dict == "model_lds_sobol.pth":
        model = DDPM_low_discrepancy(mnist_unet, T=T, sampler = "sobol").to(device)
        model.load_state_dict(torch.load(state_dict)) 
    else: 
        model = DDPM_classic(mnist_unet, T=T).to(device)
        model.load_state_dict(torch.load(state_dict))

    # Sample generation 
    generate_save_samples(model,
                          dataloader_gen,
                          root_dir = folder)
    # Fid computation  
    print(f"Evaluating {state_dict.split('.')[0]}")
    FID = compute_fid(generated_images_dir = folder,
                      evaluation_images_dir = eval_path,
                      device = "cuda")
    FIDs.append(FID)